In [ ]:
import numpy as np
"""

"""
import random



# classic implimentation of hnefatafl
class Board:
    def __init__(self):
        # black is 1, white is -1, empty is 0 -2 is the king 2 is edge
        # board is 11x11 - [x][y](column, row)
        self.board = np.array([[2,0,0,1,1,1,1,1,0,0,2],
                              [0,0,0,0,0,1,0,0,0,0,0],
                              [0,0,0,0,0,0,0,0,0,0,0],
                              [1,0,0,0,0,-1,0,0,0,0,1],
                              [1,0,0,0,-1,-1,-1,0,0,0,1],
                              [1,1,0,-1,-1,-2,-1,-1,0,1,1],
                              [1,0,0,0,-1,-1,-1,0,0,0,1],
                              [1,0,0,0,0,-1,0,0,0,0,1],
                              [0,0,0,0,0,0,0,0,0,0,0],
                              [0,0,0,0,0,1,0,0,0,0,0],
                              [2,0,0,1,1,1,1,1,0,0,2]])
    # funciton to print the board
    def print_board(self, positions_marked = []):
        column_index = 0
        # go through every column
        for column in self.board:
            row_index = 0
            if (column_index < 10):
                if (column_index != 0):
                    print(column_index, end="  ")
            else:
                # if not, just print the column index(on the side)
                print(column_index, end=" ")
                # go through every row
            for row in column:
                if (column_index == 0 and row_index == 0):
                    print("   ", end = "")
                    # print on the first line all of the numbers
                    for i in range(11):
                        print(i, end = " ")
                    print("")
                    print("0  ", end="")
                # actually printing the board based on the numbers inside.
                if (row_index, column_index) in positions_marked:
                    print("X", end = " ")
                elif row == 0:
                    # empty square
                    print("▢", end = " ")
                elif row == 1:
                    print("B", end = " ")
                elif row == -1:
                    print("W", end = " ")
                elif row == -2:
                    print("K", end = " ")
                elif row == 2:
                    # edge
                    print("▨", end = " ")


                row_index += 1
            column_index += 1
            row_index = 0
            print()

    def mask_directions(self, position, directions):
        # mask the directions that are not possible to move in (used spasifically for to know how much the king is surrounded\)
        masked_directions = []
        # loop through the four directions to move (up/down/right/left)
        for dx, dy in directions:
            # if in that direction it is not the edge of the board(one of the four sides) add it to the masked directions
            if (0 <= position[0] + dx < 11 and 0 <= position[1] + dy < 11):
                masked_directions.append((dx, dy))
        return masked_directions

    # check if a player has won, if black return 1, if white return -1, if no one return 0
    def check_winner(self):
        # if no black pieces are on the board, white wins
        if not (np.argwhere(self.board == 1).size):
            return -1

        # check if the king is in one of the four corners
        king_pos = np.argwhere(self.board == -2)
        if king_pos.size:
            kx, ky = king_pos[0]
            if (kx, ky) in [(0, 0), (0, 10), (10, 0), (10, 10)]:
                # indicate white wins
                return -1
        else:
            # indicate black wins
            return 1

        return 0

    def how_much_king_surrounded(self):
      # check how much the king is surrounded by oppisate players (black)
      directions = [(0,1), (0,-1), (1,0), (-1,0)]
      king_pos = np.argwhere(self.board == -2)
      if king_pos.size:
        directions_masked = self.mask_directions(king_pos[0], directions)
        kx, ky = king_pos[0]
        amount_surrounded = 4
        for dx, dy in directions_masked:
            nx, ny = kx + dx, ky + dy
            # if they are black, decrease one from the amoutn_surrounded(max of 4, and as little as zero)
            if not (0 <= nx < 11 and 0 <= ny < 11 and self.board[nx, ny] == 1):
                amount_surrounded -= 1
        return amount_surrounded
      # if the king is "dead" than he is fully surrounded
      return 4


    def capture_enemies(self, player, end):
        # the king can't capture, if king don't capture return -1
        if (self.board[end[0], end[1]] == -2):
            return -1
        # Define potential directions (up, down, left, right)
        directions = [(0,1), (0,-1), (1,0), (-1,0)]

        # Identify the opposing player
        opponent = -1 if player == 1 else 1

        for dx, dy in directions:
            adjacent_x = end[0] + dx
            adjacent_y = end[1] + dy
            capture_x = end[0] + 2*dx
            capture_y = end[1] + 2*dy

            if 0 <= adjacent_x < 11 and 0 <= adjacent_y < 11 and 0 <= capture_x < 11 and 0 <= capture_y < 11:
                # Capture opposing pieces
                if self.board[adjacent_x, adjacent_y] == opponent:
                    if self.board[capture_x, capture_y] == player or self.board[capture_x, capture_y] == 2 and self.board[capture_x, capture_y] != -2:
                        self.board[adjacent_x, adjacent_y] = 0
                        return 2

        # If black moved, check if the king is fully surrounded
        if player == 1:
            king_pos = np.argwhere(self.board == -2)
            if king_pos.size:
                directions_masked = self.mask_directions(king_pos[0], directions)
                kx, ky = king_pos[0]
                surrounded = True
                for dx, dy in directions_masked:
                    nx, ny = kx + dx, ky + dy
                    # if one of the masked directions doesn't have a black player on it, then the king is not surrounded
                    if not (0 <= nx < 11 and 0 <= ny < 11 and self.board[nx, ny] == 1):
                        surrounded = False
                        break
                if surrounded:
                    self.print_board()
                    self.board[kx, ky] = 0  # Remove king
                    print("king dead")
                    # signal king death
                    return 1
        # signal to the game that the king is not captured
        return -1

    def get_possible_moves_king(self, player_position = (0,0)):
        # the king can move to any free space that is not occupied by another player and is in the same row or column
        # the king can move to the edge of the board(marked with 2 in the board)
        # implement the king movement

        # the only difference between this and the regular function is that in here I don't check for edges (2)
        moves = []
        # check the left
        for i in range(player_position[0] - 1, -1, -1):
            if self.board[i][player_position[1]] == 0:
                moves.append((i, player_position[1]))
            elif self.board[i][player_position[1]] == 2:
                moves.append((i, player_position[1]))
                break
            else:
                break
        # check the right
        for i in range(player_position[0] + 1, 11):
            if self.board[i][player_position[1]] == 0:
                moves.append((i, player_position[1]))
            elif self.board[i][player_position[1]] == 2:
                moves.append((i, player_position[1]))
                break
            else:
                break
        # check the column
        # check the up
        for i in range(player_position[1] - 1, -1, -1):
            if self.board[player_position[0]][i] == 0:
                moves.append((player_position[0], i))
            elif self.board[player_position[0]][i] == 2:
                moves.append((player_position[0], i))
                break
            else:
                break
        # check the down
        for i in range(player_position[1] + 1, 11):
            if self.board[player_position[0]][i] == 0:
                moves.append((player_position[0], i))
            elif self.board[player_position[0]][i] == 2:
                moves.append((player_position[0], i))
                break
            else:
                break
        return moves


    def get_possible_moves(self, player, player_position = (0,0)):
        moves = []
        # moves dict where for each player position there is a list of possible moves
        moves_dict = {}
        # the player is the type  and the player position is the position of the player in question
        # the player can move to any free space that is not occupied by another player and is in the same row or column
        # the all players but the king can't move to the edge of the board(marked with 2 in the board)
        # the player cannot move to a space occupied by another player and all the spaces that follow that player in the same row or column

        # if the "player" is not really a player(such as an empty square) than there are no moves.
        if player not in [1, -1,-2]:
            print(player)
            return moves
        # if the king, than handel correctly
        if player == -2:
            return self.get_possible_moves_king(player_position)
        # if the player position is (0,0) than it indicates that the user wanted to get all of the possible moves for the white/black players, and not a spasific piece
        # if not, than just the moves of the player in that position. no player can get to (0,0) other than the king(in which case he wins, and therefor has no moves)
        if (player_position != (0,0)):
            # check the left
            for i in range(player_position[0] - 1, -1, -1):
                if self.board[i][player_position[1]] == 0:
                    moves.append((i, player_position[1]))
                elif self.board[i][player_position[1]] == player or self.board[i][player_position[1]] == 2:  # Changed line
                    break
                else:
                    break
            # check the right
            for i in range(player_position[0] + 1, 11):
                if self.board[i][player_position[1]] == 0:
                    moves.append((i, player_position[1]))
                elif self.board[i][player_position[1]] == player or self.board[i][player_position[1]] == 2:  # Changed line
                    break
                else:
                    break
            # check the column
            # check the up
            for i in range(player_position[1] - 1, -1, -1):
                if self.board[player_position[0]][i] == 0:
                    moves.append((player_position[0], i))
                elif self.board[player_position[0]][i] == player or self.board[player_position[0]][i] == 2:  # Changed line
                    break
                else:
                    break
            # check the down
            for i in range(player_position[1] + 1, 11):
                if self.board[player_position[0]][i] == 0:
                    moves.append((player_position[0], i))
                elif self.board[player_position[0]][i] == player or self.board[player_position[0]][i] == 2:  # Changed line
                    break
                else:
                    break

            return moves
        else:
            # get all the positions of the player with the player type
            player_positions = np.argwhere(self.board == player)
            # use this funciton, but with an actual player position
            for position in player_positions:
                moves_dict[position[0], position[1]] = []
                moves_dict[position[0],position[1]] += self.get_possible_moves(player, (position[0], position[1]))
            return moves_dict

    def get_possible_moves_train(self, player):
        # simplefied versin for the training
        moves_dict = {}
         # get all the positions of the player with the player type
        player_positions = np.argwhere(self.board == player)

        for position in player_positions:
            moves_dict[position[0], position[1]] = []
            moves_dict[position[0],position[1]] += self.get_possible_moves(player, (position[0], position[1]))
        # if the player is white, then also add the possible moves for the king
        if player == -1:
            king_positions = np.argwhere(self.board == -2)
            # if the king is on the board
            if (king_positions.size):
                #print(king_positions)
                moves_dict[king_positions[0][0], king_positions[0][1]] = []
                moves_dict[king_positions[0][0],king_positions[0][1]] += self.get_possible_moves(-2, (king_positions[0][0], king_positions[0][1]))


        return moves_dict


    def move(self, player, start, end):
        # If possible to move, than move
        if self.board[start[0], start[1]] == -2:
            if (self.board[end[0], end[1]] == 2):
                self.board[start[0], start[1]] = 0
                self.board[end[0], end[1]] = -2
                return True
        # a piece on the end position, not possible
        if (self.board[end[0], end[1]] != 0):
            return False
        if self.board[start[0], start[1]] == player or (self.board[start[0], start[1]] == -2 and player == -1):
            # check if the move with the player in the start position is possible
            if not (end in self.get_possible_moves(player, start)):
                return False
        else:
            return False
        # move the player
        if (self.board[start[0], start[1]] == -2):
            self.board[start[0], start[1]] = 0
            self.board[end[0], end[1]] = -2
            return True
        self.board[start[0], start[1]] = 0
        self.board[end[0], end[1]] = player
        return True

    def is_game_over(self):
        return self.check_winner() != 0

    def get_board(self):
        return self.board
    def reset_board(self):
        self.board = np.array([[2,0,0,1,1,1,1,1,0,0,2],
                              [0,0,0,0,0,1,0,0,0,0,0],
                              [0,0,0,0,0,0,0,0,0,0,0],
                              [1,0,0,0,0,-1,0,0,0,0,1],
                              [1,0,0,0,-1,-1,-1,0,0,0,1],
                              [1,1,0,-1,-1,-2,-1,-1,0,1,1],
                              [1,0,0,0,-1,-1,-1,0,0,0,1],
                              [1,0,0,0,0,-1,0,0,0,0,1],
                              [0,0,0,0,0,0,0,0,0,0,0],
                              [0,0,0,0,0,1,0,0,0,0,0],
                              [2,0,0,1,1,1,1,1,0,0,2]])
    def print_amount(self):
        one = len(np.argwhere(self.board != 0))
        two = len(np.argwhere(self.board == 2))
        print("The amount of pieces on the board is", one - two)

In [ ]:
import tensorflow as tf
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Dense, Flatten, Conv2D, Lambda, Input, BatchNormalization,
    Activation, Add, Dropout
)
from tensorflow.keras.optimizers import Adam
import gym
from gym import spaces
import pickle

def print_board(board, positions_marked = []):
    column_index = 0
    for column in board:
        row_index = 0
        if (column_index < 10):
            if (column_index != 0):
                print(column_index, end="  ")
        else:
            print(column_index, end=" ")
        for row in column:
            if (column_index == 0 and row_index == 0):
                print("   ", end = "")
                for i in range(11):
                    print(i, end = " ")
                print("")
                print("0  ", end="")
            if (row_index, column_index) in positions_marked:
                print("X", end = " ")
            elif row == 0:
                print("▢", end = " ")
            elif row == 1:
                print("B", end = " ")
            elif row == -1:
                print("W", end = " ")
            elif row == -2:
                print("K", end = " ")
            elif row == 2:
                print("▨", end = " ")

            row_index += 1
        column_index += 1
        row_index = 0
        print()

# create an env(environment) based on the GYM library from OpenAI. this makes the training process much more simple
class HnefataflEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.board = Board()
        # the action space
        self.action_space = spaces.Discrete(11 * 11 * 11 * 11)
        # the size of the board
        self.observation_space = spaces.Box(low=-2, high=2, shape=(11, 11), dtype=np.int32)
        self.previous_board = None
        self.previous_action = None
        self.flag = False
        self.prev_amount = 0

    def valid_moves_mask(self):
        # mas the valid moves only, is a huge list in the size of the actionspace, that can represent all legal moves in the game.
        # every move (action) that is "masked" (has true in it's index) is considered legal.
        mask = np.zeros(self.action_space.n, dtype=bool)
        valid_moves = self.board.get_possible_moves_train(self.current_player)
        for start_pos, end_positions in valid_moves.items():
            start_x, start_y = start_pos
            for end_pos in end_positions:
                end_x, end_y = end_pos
                action = (start_x * 11 * 11 * 11) + (start_y * 11 * 11) + (end_x * 11) + end_y
                mask[action] = True
        return mask

    def step(self, action):
        # find the actions start and end.
        start_x = action // (11 * 11 * 11)
        start_y = (action % (11 * 11 * 11)) // (11 * 11)
        end_x = (action % (11 * 11)) // 11
        end_y = action % 11
        current_player = self.current_player

        self.previous_board = self.board.get_board()
        self.previous_action = [(start_x, start_y), (end_x, end_y)]

        valid = self.board.move(current_player, (start_x, start_y), (end_x, end_y))

        """
        if (valid and (end_x, end_y) in [(10, 0), (0, 10), (10, 10), (0, 0)]):
            print("winning move and legal")
        """
        captured = self.board.capture_enemies(current_player, (end_x, end_y))

        winner = self.board.check_winner()
        done = bool(winner)


        # calculate reward, changes between runs.
        reward = 0
        if (not (-1 in self.board.get_board())) and current_player == 1 and self.flag == False:
            reward = 10
            self.flag = True
        elif captured == 2 and current_player == 1:
            reward = 3
        elif (not (-1 in self.board.get_board())) and self.flag == True:
            reward = 0.3
        amount = self.board.how_much_king_surrounded()
        if (amount < self.prev_amount):
          reward -= 12
        if (amount == 1):
          reward += 0.5
        if (amount == 2):
          reward += 1.2
        if (amount == 3):
          reward += 2
        self.prev_amount = amount


        if done:
            self.flag = False
            # give high reward if the agent wins
            reward = 400 if (winner == 1 and current_player == 1) or (winner == -1 and current_player == -1) else -400

        # switch players
        self.current_player *= -1
        return self.board.get_board(), reward, done, {}

    def reset(self):
        self.board.reset_board()
        self.current_player = 1
        return self.board.get_board()

class DQNAgent:
    def __init__(self, state_size, action_size, player):
        self.state_size = state_size
        self.action_size = action_size
        self.player = player
        self.memory = deque(maxlen=50000)  # Default memory buffer for the replay
        self.priorities = deque(maxlen=50000)  # Priority buffer for PER
        self.gamma = 0.95
        self.epsilon = 0.99
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.998
        self.learning_rate = 0.001
        self.tau = 0.01  # For soft updates
        self.alpha = 0.6  # PER exponent
        self.beta = 0.4   # PER importance sampling factor
        self.beta_increment = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def save_weights(self, filename):
        self.model.save_weights(filename)

    def load_weights(self, filename):
        self.model.load_weights(filename)


    def save_model(self, filename):
        self.model.save(filename)

    def load_model(self, filename):
        # the model has a custom function inside of it, you have to mention it when loading the model
        # this is for the dualing dqn part of the model
        custom_objects = {'<lambda>': lambda a: a[0] + a[1] - tf.reduce_mean(a[1], axis=1, keepdims=True)}
        self.model = tf.keras.models.load_model(filename,safe_mode=False,custom_objects=custom_objects)
        self.target_model = tf.keras.models.load_model(filename,safe_mode=False,custom_objects=custom_objects)

    def _build_model(self):
      inputs = Input(shape=(11, 11, 1))
      # regualr convolutional network
      # the theory is that the board is somewhat like a picture, and that certain patterns might appear that can help the model learn
      # which actions are the best. For example, if the king is about to be fully surounded, a filter might be able to find that, and indicate
      # that completing the encercinment might be very good(winning the game)
      x = Conv2D(256, (5, 5), activation='relu', padding='same')(inputs)
      x = Conv2D(128, (5, 5), activation='relu')(x)
      x = Conv2D(64, (3, 3), activation='relu')(x)
      x = Conv2D(32, (3, 3), activation='relu')(x)
      x = Flatten()(x)

      # Dueling architecture
      advantage = Dense(512, activation='relu')(x)
      advantage = Dropout(0.5)(advantage)  # Add Dropout for regularization
      advantage = Dense(256, activation='relu')(advantage)
      advantage = Dense(self.action_size)(advantage)

      # Value Stream
      value = Dense(512, activation='relu')(x)
      value = Dropout(0.5)(value)  # Add Dropout for regularization
      value = Dense(256, activation='relu')(value)
      value = Dense(1)(value)

      # Add residual connections (skip connections) to both streams
      advantage_residual = Add()([advantage, Dense(self.action_size)(x)])
      value_residual = Add()([value, Dense(1)(x)])

      # Combine value and advantage
      # Provide an explicit output_shape
      q_values = Lambda(lambda a: a[0] + a[1] - tf.reduce_mean(a[1], axis=1, keepdims=True),output_shape=(self.action_size,)  # Explicitly specify the output shape
)([value, advantage])


      model = Model(inputs, q_values)
      # compile the model with adam and mse(mean squared error)
      model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate, clipnorm=1.0))
      return model

    def update_target_model(self):
        # Soft update the target network
        # just copying the weights
        model_weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        new_weights = [
            self.tau * mw + (1 - self.tau) * tw
            for mw, tw in zip(model_weights, target_weights)
        ]
        self.target_model.set_weights(new_weights)

    def remember(self, state, action, reward, next_state, done):
        # add to the replay memory an experience that happned
        max_priority = max(self.priorities, default=1.0)
        self.memory.append((state, action, reward, next_state, done))
        self.priorities.append(max_priority)

    def _get_priority(self, error):
        # calculate priority
        return (abs(error) + 1e-5) ** self.alpha

    def replay(self, batch_size):
      # if there isn't enough examples in the memory
      if len(self.memory) < batch_size:
          return

      # Compute priorities and sample
      priorities = np.array(self.priorities)
      probabilities = priorities ** self.alpha
      probabilities /= probabilities.sum()

      indices = np.random.choice(len(self.memory), batch_size, p=probabilities)
      minibatch = [self.memory[i] for i in indices]
      is_weights = (len(self.memory) * probabilities[indices]) ** -self.beta
      is_weights /= is_weights.max()

      # Increment beta
      self.beta = min(1.0, self.beta + self.beta_increment)

      states = np.array([experience[0] for experience in minibatch])
      actions = np.array([experience[1] for experience in minibatch])
      rewards = np.array([experience[2] for experience in minibatch])
      next_states = np.array([experience[3] for experience in minibatch])
      dones = np.array([experience[4] for experience in minibatch])

      # get the predicted q-values. used for the double dqn.
      # predicting the max-value from the main(active) network, and getting the values from the target.
      q_values = self.model.predict(states, verbose=0)
      next_q_values_online = self.model.predict(next_states, verbose=0)
      next_q_values_target = self.target_model.predict(next_states, verbose=0)

      # double dqn
      for i, index in enumerate(indices):
          # Ensure `target` is assigned in all cases
          target = q_values[i][actions[i]]  # Default value (should not be used in practice)
          if dones[i]:
              target = rewards[i]
          else:
              best_action = np.argmax(next_q_values_online[i])
              target = rewards[i] + self.gamma * next_q_values_target[i][best_action]

          # calculate td error for optimized replay
          td_error = target - q_values[i][actions[i]]
          q_values[i][actions[i]] = target

          # Update priority
          self.priorities[index] = self._get_priority(td_error)

      # "teach" the model
      self.model.fit(states, q_values, sample_weight=is_weights, epochs=1, verbose=0)
      # decay epsilon
      self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def act(self, state, valid_moves, exploit=False):
        # choose an action based on the epsilon greedy policy.
        if random.random() <= self.epsilon and not exploit:

            valid_indices = [i for i, valid in enumerate(valid_moves) if valid]
            return random.choice(valid_indices)

        state = np.expand_dims(state, axis=0)
        q_values = self.model.predict(state, verbose=0)[0]
        q_values = np.where(valid_moves, q_values, -np.inf)
        return np.argmax(q_values)




env = HnefataflEnv()
state_size = (11, 11)
action_size = 11 * 11 * 11 * 11
load_prev = input("load previous run? (y/n): ")
if load_prev == 'y':
    with open('black_agent_two.pickle', 'rb') as handle:
        black_agent = pickle.load(handle)
    #with open('white_agent_two.pickle', 'rb') as handle:
     #   white_agent = pickle.load(handle)
else:
    black_agent = DQNAgent(state_size, action_size, 1)
    #black_agent.load_weights('black_weights_600.weights.h5')
    #black_agent.epsilon = 0.7
    white_agent = DQNAgent(state_size, action_size, -1)

batch_size = 512
n_episodes = 12000
without_memory = 10
reward_history = []

for episode in range(n_episodes):

    state = env.reset()
    done = False
    turn_count = 0

    current_reward = 0
    while not done:
        turn_count += 1

        valid_moves = env.valid_moves_mask()
        action = black_agent.act(state, valid_moves)
        previous_action_black = action
        prev_state = state
        next_state, reward, done, _ = env.step(action)
        reward_black = rewardנקקאש
        black_agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            current_reward += reward
            black_agent.remember(state, action, reward, next_state, done)
            #white_agent.remember(prev_state, previous_action_white, -reward, next_state, done)
            print("black wins!")
            print_board(env.previous_board)
            print(env.previous_action)
            break

        valid_moves = env.valid_moves_mask()
        action = white_agent.act(state, valid_moves)
        prev_state = state
        next_state, reward, done, _ = env.step(action)
        #white_agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            current_reward += -(reward*2.5)
            black_agent.remember(prev_state, previous_action_black,-(reward*2.5) , next_state, done)
            print("white wins!")
            print_board(env.previous_board)
            print(env.previous_action)
            break
        else:
            current_reward += reward_black
            black_agent.remember(prev_state, previous_action_black, reward_black, next_state, done)
        if (turn_count % 3000 == 0):
            env.board.print_board()
            print("turn count: ", {turn_count})


    if episode > without_memory:
        print("replay")
        black_agent.replay(batch_size)
        #white_agent.replay(batch_size)

    if episode % 50 == 0:
        black_agent.update_target_model()
        #white_agent.update_target_model()
    if episode % 40 == 0:
        black_agent.save_weights('black_weights.weights.h5')
        #white_agent.save_model('white_model_two.keras')
        black_agent.save_model('black_model_two.keras')
    if episode % 100 == 0:
        epsilon_file_name = f"black_agent_epsilon_{episode}.pickle"
        with open(epsilon_file_name, 'wb') as handle:
            pickle.dump(black_agent.epsilon, handle, protocol=pickle.HIGHEST_PROTOCOL)
        place = f"black_weights_{episode}.weights.h5"
        black_agent.save_weights(place)
        #with open('white_agent_two.pickle', 'wb') as handle:
        #    pickle.dump(white_agent, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('reward_history_black.pickle', 'wb') as handle:
            pickle.dump(reward_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

    if episode % 1 == 0:
        reward_history.append(current_reward)
        print(f"Episode: {episode}, Total turns: {turn_count}, Epsilon: {black_agent.epsilon}, reward: {current_reward}")

black_agent.model.save('black_model.h5')
white_agent.model.save('white_model.h5')



load previous run? (y/n): n
white wins!
   0 1 2 3 4 5 6 7 8 9 10 
0  K ▢ B ▢ ▢ ▢ B B ▢ ▢ ▨ 
1  ▢ ▢ ▢ ▢ ▢ B W ▢ ▢ B ▢ 
2  ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ ▢ ▢ 
3  ▢ ▢ ▢ B B B B ▢ B ▢ ▢ 
4  ▢ ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ ▢ 
5  ▢ ▢ B B ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
6  ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ B ▢ W 
7  ▢ ▢ B ▢ ▢ ▢ ▢ B W ▢ W 
8  ▢ B W B ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
9  ▢ ▢ ▢ W B ▢ ▢ ▢ ▢ ▢ ▢ 
10 ▨ ▢ ▢ ▢ ▢ B ▢ B ▢ ▢ ▨ 
[(6, 0), (0, 0)]
Episode: 0, Total turns: 149, Epsilon: 0.99, reward: -961.5
white wins!
   0 1 2 3 4 5 6 7 8 9 10 
0  ▨ ▢ ▢ ▢ ▢ ▢ B B ▢ ▢ ▨ 
1  ▢ ▢ ▢ B ▢ B B ▢ ▢ ▢ ▢ 
2  ▢ ▢ ▢ B ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
3  ▢ ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ ▢ 
4  ▢ ▢ W ▢ ▢ B ▢ B ▢ B ▢ 
5  ▢ ▢ W ▢ B B ▢ ▢ B ▢ W 
6  ▢ ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ ▢ 
7  ▢ W B B B ▢ ▢ ▢ ▢ ▢ ▢ 
8  ▢ ▢ B ▢ ▢ ▢ ▢ ▢ ▢ ▢ B 
9  ▢ ▢ B B ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
10 K ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▨ 
[(6, 0), (10, 0)]
Episode: 1, Total turns: 165, Epsilon: 0.99, reward: -964.7
white wins!
   0 1 2 3 4 5 6 7 8 9 10 
0  ▨ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▨ 
1  ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ B 
2  ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ B ▢ 
3  ▢ B ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
4  ▢ W ▢ ▢ W ▢ ▢ ▢ B B ▢

KeyboardInterrupt: 

In [ ]:
# try to train the model again, using the same weights, but changing the reward system. first, giving A lot extra reward for "killing" (surrounding) the king when he is not in the middle square.
# in addition, giving a higher chance for the king to move in relation to the other moves - thus technically making the white agent not completely random.

import tensorflow as tf
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Dense, Flatten, Conv2D, Lambda, Input, BatchNormalization,
    Activation, Add, Dropout
)
from tensorflow.keras.optimizers import Adam
import gym
from gym import spaces
import pickle

def print_board(board, positions_marked = []):
    column_index = 0
    for column in board:
        row_index = 0
        if (column_index < 10):
            if (column_index != 0):
                print(column_index, end="  ")
        else:
            print(column_index, end=" ")
        for row in column:
            if (column_index == 0 and row_index == 0):
                print("   ", end = "")
                for i in range(11):
                    print(i, end = " ")
                print("")
                print("0  ", end="")
            if (row_index, column_index) in positions_marked:
                print("X", end = " ")
            elif row == 0:
                print("▢", end = " ")
            elif row == 1:
                print("B", end = " ")
            elif row == -1:
                print("W", end = " ")
            elif row == -2:
                print("K", end = " ")
            elif row == 2:
                print("▨", end = " ")

            row_index += 1
        column_index += 1
        row_index = 0
        print()


class HnefataflEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.board = Board()
        self.action_space = spaces.Discrete(11 * 11 * 11 * 11)
        self.observation_space = spaces.Box(low=-2, high=2, shape=(11, 11), dtype=np.int32)
        self.previous_board = None
        self.previous_action = None
        self.flag = False
        self.prev_amount = 0

    def valid_moves_mask(self):
        mask = np.zeros(self.action_space.n, dtype=bool)
        valid_moves = self.board.get_possible_moves_train(self.current_player)
        for start_pos, end_positions in valid_moves.items():
            start_x, start_y = start_pos
            for end_pos in end_positions:
                end_x, end_y = end_pos
                action = (start_x * 11 * 11 * 11) + (start_y * 11 * 11) + (end_x * 11) + end_y
                mask[action] = True
        return mask


    def step(self, action):
        start_x = action // (11 * 11 * 11)
        start_y = (action % (11 * 11 * 11)) // (11 * 11)
        end_x = (action % (11 * 11)) // 11
        end_y = action % 11
        current_player = self.current_player
        king_pos = np.argwhere(self.board.get_board() == -2)

        self.previous_board = self.board.get_board()
        self.previous_action = [(start_x, start_y), (end_x, end_y)]

        valid = self.board.move(current_player, (start_x, start_y), (end_x, end_y))
        if (valid and (end_x, end_y) in [(10, 0), (0, 10), (10, 10), (0, 0)]):
            print("winning move and legal")



        captured = self.board.capture_enemies(current_player, (end_x, end_y))

        if not valid:
            return self.board.get_board(), -10, False, {}

        winner = self.board.check_winner()
        done = bool(winner)

        reward = 0
        if (not (-1 in self.board.get_board())) and current_player == 1 and self.flag == False:
            reward = 10
            self.flag = True
        elif captured == 2 and current_player == 1:
            reward = 3
        elif (not (-1 in self.board.get_board())) and self.flag == True:
            reward = 0.3
        amount = self.board.how_much_king_surrounded()
        if (amount < self.prev_amount):
          reward -= 15
        if (amount == 1):
          reward += 0.3
        if (amount == 2):
          reward += 1
        if (amount == 3):
          reward += 1.5
        self.prev_amount = amount


        if done:
            self.flag = False
            reward = 800 if (winner == 1 and current_player == 1) or (winner == -1 and current_player == -1) else -400


            if (king_pos.size):
              if (current_player == 1 and (king_pos[0][0] ,king_pos[0][1]) != (5,5)):
                reward = 2500 if (winner == 1 and current_player == 1) or (winner == -1 and current_player == -1) else -400


        self.current_player *= -1
        return self.board.get_board(), reward, done, {}

    def reset(self):
        self.board.reset_board()
        self.current_player = 1
        return self.board.get_board()

class DQNAgent:
    def __init__(self, state_size, action_size, player):
        self.state_size = state_size
        self.action_size = action_size
        self.player = player
        self.memory = deque(maxlen=50000)  # Default memory buffer
        self.priorities = deque(maxlen=50000)  # Priority buffer for PER
        self.gamma = 0.95
        self.epsilon = 0.99
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.learning_rate = 0.001
        self.tau = 0.01  # For soft updates
        self.alpha = 0.6  # PER exponent
        self.beta = 0.4   # PER importance sampling factor
        self.beta_increment = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def save_weights(self, filename):
        self.model.save_weights(filename)

    def load_weights(self, filename):
        self.model.load_weights(filename)


    def save_model(self, filename):
        self.model.save(filename)

    def load_model(self, filename):
        custom_objects = {'<lambda>': lambda a: a[0] + a[1] - tf.reduce_mean(a[1], axis=1, keepdims=True)}
        self.model = tf.keras.models.load_model(filename,safe_mode=False,custom_objects=custom_objects)
        self.target_model = tf.keras.models.load_model(filename,safe_mode=False,custom_objects=custom_objects)

    def _build_model(self):
      inputs = Input(shape=(11, 11, 1))
      x = Conv2D(256, (5, 5), activation='relu', padding='same')(inputs)
      x = Conv2D(128, (5, 5), activation='relu')(x)
      x = Conv2D(64, (3, 3), activation='relu')(x)
      x = Conv2D(32, (3, 3), activation='relu')(x)
      x = Flatten()(x)

      # Dueling architecture
      advantage = Dense(512, activation='relu')(x)
      advantage = Dropout(0.5)(advantage)  # Add Dropout for regularization
      advantage = Dense(256, activation='relu')(advantage)
      advantage = Dense(self.action_size)(advantage)

      # Value Stream
      value = Dense(512, activation='relu')(x)
      value = Dropout(0.5)(value)  # Add Dropout for regularization
      value = Dense(256, activation='relu')(value)
      value = Dense(1)(value)

      # Add residual connections (skip connections) to both streams
      advantage_residual = Add()([advantage, Dense(self.action_size)(x)])
      value_residual = Add()([value, Dense(1)(x)])

      # Combine value and advantage
      # Provide an explicit output_shape
      q_values = Lambda(lambda a: a[0] + a[1] - tf.reduce_mean(a[1], axis=1, keepdims=True),output_shape=(self.action_size,)  # Explicitly specify the output shape
)([value, advantage])


      model = Model(inputs, q_values)
      model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate, clipnorm=1.0))
      return model

    def update_target_model(self):
        # Soft update the target network
        model_weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        new_weights = [
            self.tau * mw + (1 - self.tau) * tw
            for mw, tw in zip(model_weights, target_weights)
        ]
        self.target_model.set_weights(new_weights)

    def remember(self, state, action, reward, next_state, done):
        max_priority = max(self.priorities, default=1.0)
        self.memory.append((state, action, reward, next_state, done))
        self.priorities.append(max_priority)

    def _get_priority(self, error):
        return (abs(error) + 1e-5) ** self.alpha

    def replay(self, batch_size):
      if len(self.memory) < batch_size:
          return

      # Compute priorities and sample
      priorities = np.array(self.priorities)
      probabilities = priorities ** self.alpha
      probabilities /= probabilities.sum()

      indices = np.random.choice(len(self.memory), batch_size, p=probabilities)
      minibatch = [self.memory[i] for i in indices]
      is_weights = (len(self.memory) * probabilities[indices]) ** -self.beta
      is_weights /= is_weights.max()

      # Increment beta
      self.beta = min(1.0, self.beta + self.beta_increment)

      states = np.array([experience[0] for experience in minibatch])
      actions = np.array([experience[1] for experience in minibatch])
      rewards = np.array([experience[2] for experience in minibatch])
      next_states = np.array([experience[3] for experience in minibatch])
      dones = np.array([experience[4] for experience in minibatch])

      q_values = self.model.predict(states, verbose=0)
      next_q_values_online = self.model.predict(next_states, verbose=0)
      next_q_values_target = self.target_model.predict(next_states, verbose=0)

      for i, index in enumerate(indices):
          # Ensure `target` is assigned in all cases
          target = q_values[i][actions[i]]  # Default value (should not be used in practice)
          if dones[i]:
              target = rewards[i]
          else:
              best_action = np.argmax(next_q_values_online[i])
              target = rewards[i] + self.gamma * next_q_values_target[i][best_action]

          td_error = target - q_values[i][actions[i]]
          q_values[i][actions[i]] = target

          # Update priority
          self.priorities[index] = self._get_priority(td_error)

      self.model.fit(states, q_values, sample_weight=is_weights, epochs=1, verbose=0)
      self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def act(self, state, valid_moves, exploit=False):
        if random.random() <= self.epsilon and not exploit:
            valid_indices = [i for i, valid in enumerate(valid_moves) if valid]
            return random.choice(valid_indices)

        state = np.expand_dims(state, axis=0)
        q_values = self.model.predict(state, verbose=0)[0]
        q_values = np.where(valid_moves, q_values, -np.inf)
        return np.argmax(q_values)


    def act_weighted(self, state, valid_moves, board, weighted_value=1, exploit=False):
      valid_indices = [i for i, valid in enumerate(valid_moves) if valid]

      # Find king position
      king_pos = np.argwhere(board == -2)
      if king_pos.size:
          king_moves = board.get_possible_moves_king(tuple(king_pos[0]))

          # Compute indices efficiently
          index_factors = np.array([11**3, 11**2, 11, 1])
          indexs = [(np.array(start + end) * index_factors).sum() for start, end in king_moves]

          # Extend valid indices with weighted king move indices
          valid_indices.extend(index for _ in range(weighted_value) for index in indexs)

      return random.choice(valid_indices)



env = HnefataflEnv()
state_size = (11, 11)
action_size = 11 * 11 * 11 * 11
load_prev = input("load previous run? (y/n): ")
if load_prev == 'y':
    with open('black_agent_two.pickle', 'rb') as handle:
        black_agent = pickle.load(handle)
    #with open('white_agent_two.pickle', 'rb') as handle:
     #   white_agent = pickle.load(handle)
else:
    black_agent = DQNAgent(state_size, action_size, 1)
    black_agent.load_weights('black_weights_3000.weights.h5')
    black_agent.epsilon = 0.99
    white_agent = DQNAgent(state_size, action_size, -1)

batch_size = 512
n_episodes = 12000
without_memory = 10
reward_history = []

for episode in range(n_episodes):

    state = env.reset()
    done = False
    turn_count = 0

    current_reward = 0
    while not done:
        turn_count += 1

        valid_moves = env.valid_moves_mask()
        action = black_agent.act(state, valid_moves)
        previous_action_black = action
        prev_state = state
        next_state, reward, done, _ = env.step(action)
        reward_black = reward
        black_agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            current_reward += reward
            black_agent.remember(state, action, reward, next_state, done)
            #white_agent.remember(prev_state, previous_action_white, -reward, next_state, done)
            print("black wins!")
            print_board(env.previous_board)
            print(env.previous_action)
            break

        valid_moves = env.valid_moves_mask()
        # 2x the possibility of king movement
        action = white_agent.act_weighted(state, valid_moves, env.board, 1)
        prev_state = state
        next_state, reward, done, _ = env.step(action)
        #white_agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            current_reward += -(reward/6)
            black_agent.remember(prev_state, previous_action_black,-(reward/6) , next_state, done)
            print("white wins!")
            print_board(env.previous_board)
            print(env.previous_action)
            break
        else:
            current_reward += reward_black
            black_agent.remember(prev_state, previous_action_black, reward_black, next_state, done)
        if (turn_count % 3000 == 0):
            env.board.print_board()
            print("turn count: ", {turn_count})


    if episode > without_memory:
        print("replay")
        black_agent.replay(batch_size)
        #white_agent.replay(batch_size)

    if episode % 50 == 0:
        black_agent.update_target_model()
        #white_agent.update_target_model()
    if episode % 40 == 0:
        black_agent.save_weights('black_weights.weights.h5')
        #white_agent.save_model('white_model_two.keras')
        black_agent.save_model('black_model_two.keras')
    if episode % 100 == 0:
        epsilon_file_name = f"black_agent_epsilon_{episode}.pickle"
        with open(epsilon_file_name, 'wb') as handle:
            pickle.dump(black_agent.epsilon, handle, protocol=pickle.HIGHEST_PROTOCOL)
        place = f"black_weights_{episode}.weights.h5"
        black_agent.save_weights(place)
        #with open('white_agent_two.pickle', 'wb') as handle:
        #    pickle.dump(white_agent, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('reward_history_black.pickle', 'wb') as handle:
            pickle.dump(reward_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

    if episode % 1 == 0:
        reward_history.append(current_reward)
        print(f"Episode: {episode}, Total turns: {turn_count}, Epsilon: {black_agent.epsilon}, reward: {current_reward}")

black_agent.model.save('black_model.h5')
white_agent.model.save('white_model.h5')



/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Streaming output truncated to the last 5000 lines.
7  B ▢ ▢ ▢ ▢ ▢ W W ▢ ▢ B 
8  W ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
9  ▢ ▢ ▢ ▢ W B ▢ ▢ ▢ ▢ ▢ 
10 ▨ ▢ ▢ B B B B B ▢ ▢ ▨ 
[(5, 9), (5, 6)]
replay
Episode: 4347, Total turns: 28, Epsilon: 0.012916880171186978, reward: 801.6
winning move and legal
white wins!
   0 1 2 3 4 5 6 7 8 9 10 
0  ▨ W ▢ B ▢ ▢ B B ▢ ▢ K 
1  ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ W ▢ 
2  W ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
3  B ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ B 
4  ▢ ▢ W ▢ W B W ▢ ▢ ▢ B 
5  B ▢ B ▢ ▢ ▢ B ▢ ▢ ▢ B 
6  B ▢ ▢ ▢ ▢ B ▢ ▢ ▢ ▢ B 
7  B ▢ ▢ ▢ ▢ ▢ W ▢ ▢ ▢ B 
8  ▢ ▢ ▢ ▢ W ▢ ▢ ▢ ▢ W ▢ 
9  W ▢ ▢ W ▢ W ▢ ▢ ▢ ▢ ▢ 
10 ▨ ▢ ▢ B B B B B ▢ ▢ ▨ 
[(1, 10), (0, 10)]
replay
Episode: 4348, Total turns: 183, Epsilon: 0.01290396329101579, reward: -114.03333333333332
winning move and legal
white wins!
   0 1 2 3 4 5 6 7 8 9 10 
0  ▨ W ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▨ 
1  ▢ W ▢ ▢ ▢ ▢ W W ▢ ▢ ▢ 
2  W B B B ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
3  ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
4  ▢ ▢ ▢ ▢ ▢ ▢ B ▢ ▢ ▢ W 
5  ▢ ▢ ▢ ▢ B B B ▢ ▢ ▢ W 
6  ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
7  B ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ ▢ 
8  ▢ ▢ ▢ 

KeyboardInterrupt: 